####
다중 분류라 타겟값 원핫 해야함

In [1]:
import numpy as np
import pandas as pd 

In [49]:
%%time
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Dense, Input, InputLayer, Add, BatchNormalization, Dropout

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
import random

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
import datetime
import math

CPU times: total: 0 ns
Wall time: 0 ns


In [50]:
## 파일 생성 및 경로
import os
import datetime

def file_make(filename):
    now = datetime.datetime.now()
    date =  now.strftime('%Y-%m-%d')

    file_path = './'+ filename +'/' + date
    os.makedirs(file_path,exist_ok=True)
    print(file_path)
    return file_path

In [54]:
import pandas as pd

trn_data = pd.read_csv('./data/2022-12-14/trn_data.csv')
tst_data = pd.read_csv('./data/2022-12-14/tst_data.csv')
sub = pd.read_csv('./data/sample_submission.csv')

In [52]:
def cat_to_num(df):
    df['class'] = df['class'].replace({'A':0,'B':1,'C':2})
    return df

def num_to_cat(df):
    df['class'] = df['class'].replace({0:'A',1:'B',2:'C'})
    return df

In [57]:
trn_data = cat_to_num(trn_data)

In [59]:
remove = ['id', 
          'class', 
         ]
features = [feat for feat in trn_data.columns if feat not in remove]

### 모델

In [60]:
%%time
drop_out_pct = 0.45
l2 = 80e-6

def nn_model():
    '''
    '''
    
    activation_func = 'swish'
    inputs = Input(shape = (len(features)))
    
    x = Dense(1024, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(l2), 
              activation = activation_func)(inputs)
    
    x = Dropout(drop_out_pct)(x)
    x = BatchNormalization()(x)
    
    
    x = Dense(512, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(l2), 
              activation = activation_func)(x)
    
    x = Dropout(drop_out_pct)(x)
    x = BatchNormalization()(x)

    x = Dense(128, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(l2), 
              activation = activation_func)(x)

    x = Dropout(drop_out_pct)(x)
    x = BatchNormalization()(x)


    x = Dense(1 , 
              #use_bias  = True, 
              #kernel_regularizer = tf.keras.regularizers.l2(30e-6),
              activation = 'sigmoid')(x)
    
    model = Model(inputs, x)
    
    return model

CPU times: total: 0 ns
Wall time: 0 ns


In [61]:
%%time
architecture = nn_model()
architecture.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 19)]              0         
                                                                 
 dense_24 (Dense)            (None, 1024)              20480     
                                                                 
 dropout_18 (Dropout)        (None, 1024)              0         
                                                                 
 batch_normalization_18 (Bat  (None, 1024)             4096      
 chNormalization)                                                
                                                                 
 dense_25 (Dense)            (None, 512)               524800    
                                                                 
 dropout_19 (Dropout)        (None, 512)               0         
                                                           

In [62]:
%%time
# Defining model parameters...
BATCH_SIZE         = 128
EPOCHS             = 512 
EPOCHS_COSINEDECAY = 512 
DIAGRAMS           = True
USE_PLATEAU        = True
INFERENCE          = False
VERBOSE            = 0 
TARGET             = 'class'

CPU times: total: 0 ns
Wall time: 0 ns


In [63]:
%%time
# Defining model training function...
def fit_model(X_train, y_train, X_val, y_val, run = 0):
    '''
    '''
    lr_start = 0.1
    start_time = datetime.datetime.now()
    
    scaler = StandardScaler()

    X_train = scaler.fit_transform(X_train)

    epochs = EPOCHS    
    lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.7, patience = 4, verbose = VERBOSE)
    es = EarlyStopping(monitor = 'val_loss',patience = 48, verbose = 1, mode = 'min', restore_best_weights = True)
    tm = tf.keras.callbacks.TerminateOnNaN()
    callbacks = [lr, es, tm]
    
    # Cosine Learning Rate Decay
    if USE_PLATEAU == False:
        epochs = EPOCHS_COSINEDECAY
        lr_end = 0.0002

        def cosine_decay(epoch):
            if epochs > 1:
                w = (1 + math.cos(epoch / (epochs - 1) * math.pi)) / 2
            else:
                w = 1
            return w * lr_start + (1 - w) * lr_end
        
        lr = LearningRateScheduler(cosine_decay, verbose = 0)
        callbacks = [lr, tm]
        
    model = nn_model()
    
    optimizer_func = tf.keras.optimizers.Adam(learning_rate = lr_start)
    loss_func = tf.keras.losses.CategoricalCrossentropy()
    model.compile(optimizer = optimizer_func, loss = loss_func)
    
    X_val = scaler.transform(X_val)
    validation_data = (X_val, y_val)
    
    history = model.fit(X_train, 
                        y_train, 
                        validation_data = validation_data, 
                        epochs          = epochs,
                        verbose         = VERBOSE,
                        batch_size      = BATCH_SIZE,
                        shuffle         = True,
                        callbacks       = callbacks
                       )
    
    history_list.append(history.history)
    print(f'Training loss:{history_list[-1]["loss"][-1]:.3f}')
    callbacks, es, lr, tm, history = None, None, None, None, None
    
    
    y_val_pred = model.predict(X_val, batch_size = BATCH_SIZE, verbose = VERBOSE)
    y_val_pred = [1 if x > 0.5 else 0 for x in y_val_pred]
    
    score = accuracy_score(y_val, y_val_pred)
    print(f'Fold {run}.{fold} | {str(datetime.datetime.now() - start_time)[-12:-7]}'
          f'| ACC: {score:.5f}')
    
    score_list.append(score)
    
    tst_data_scaled = scaler.transform(tst_data[features])
    tst_pred = model.predict(tst_data_scaled)
    predictions.append(tst_pred)
    
    return model

CPU times: total: 0 ns
Wall time: 0 ns


In [76]:
%%time
# Create empty lists to store NN information...
history_list = []
score_list   = []
predictions  = []

# Define kfolds for training purposes...
kf = KFold(n_splits = 5)

for fold, (trn_idx, val_idx) in enumerate(kf.split(trn_data)):
    X_train, X_val = trn_data.iloc[trn_idx][features], trn_data.iloc[val_idx][features]
    y_train, y_val = trn_data.iloc[trn_idx][TARGET], trn_data.iloc[val_idx][TARGET]
    
    fit_model(X_train, y_train, X_val, y_val)
    
print(f'OOF AUC: {np.mean(score_list):.5f}')

Restoring model weights from the end of the best epoch: 1.
Epoch 49: early stopping
Training loss:7.213
Fold 0.0 | 00:03| ACC: 0.77358
6/6 [==============================] - 0s 2ms/step
Restoring model weights from the end of the best epoch: 1.
Epoch 49: early stopping
Training loss:7.143
Fold 0.1 | 00:03| ACC: 0.62264
6/6 [==============================] - 0s 2ms/step
Restoring model weights from the end of the best epoch: 1.
Epoch 49: early stopping
Training loss:7.373
Fold 0.2 | 00:03| ACC: 0.67308
6/6 [==============================] - 0s 2ms/step
Restoring model weights from the end of the best epoch: 1.
Epoch 49: early stopping
Training loss:6.716
Fold 0.3 | 00:03| ACC: 0.61538
6/6 [==============================] - 0s 2ms/step
Restoring model weights from the end of the best epoch: 1.
Epoch 49: early stopping
Training loss:7.306
Fold 0.4 | 00:03| ACC: 0.75000
6/6 [==============================] - 0s 2ms/step
OOF AUC: 0.68694
CPU times: total: 12.6 s
Wall time: 15.9 s


In [65]:
# %%time
# # Populated the prediction on the submission dataset and creates an output file
# sub['Transported'] = np.array(predictions).mean(axis = 0)
# sub['Transported'] = np.where(sub['Transported'] > 0.5, True, False)
# sub.to_csv('submission_nn01.csv', index = False)

In [79]:
sub = pd.read_csv('./data/sample_submission.csv')
sub = sub.drop(['class'],axis=1)

In [81]:
tst_data['class'] = np.array(predictions).mean(axis = 0)


In [83]:
submit = pd.merge(sub, tst_data[['id','class']], how='outer', on = 'id')

In [ ]:
submit['class'] = np.where(submit['class'] > 0.5, True, False)

In [ ]:
submit = num_to_cat(submit)

In [72]:
number = '01'
submit_path = file_make('submit_file') +'/submitionDNN' + number +'.csv'
submit.to_csv(submit_path,index=False)

./submit_file/2022-12-14


In [73]:
submit_check = pd.read_csv(submit_path)
submit_check

,id,class
0,TEST_000,False
1,TEST_001,True
2,TEST_002,True
3,TEST_003,True
4,TEST_004,False
...,...,...
170,TEST_170,True
171,TEST_171,True
172,TEST_172,True
173,TEST_173,True


####
다중 분류라 타겟값 원핫 해야함